### Splitting Leavetimes Dataset
- The leavetimes dataset is too large to look at at once. 
- We will split the leavetimes dataset by trip and then clean that dataframe

### Setup workspace

In [1]:
# import required packages
import pandas as pd
import numpy as np
import sqlite3
import pymysql
from datetime import datetime
import time
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.tree import export_graphviz
import scipy.stats as ss
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Functions to clean the dfs

In [2]:
# Function to clean the Leavetimes df
# Based off of the CleaningNotebook cleaning methods
def clean_leavetimes(x):
    
    categorical_columns = x[["TRIPID","PROGRNUMBER","STOPPOINTID","PLANNEDTIME_ARR", 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR','ACTUALTIME_DEP','VEHICLEID']].columns
    # Convert data type to category for these columns
    for column in categorical_columns:
        x[column] = x[column].astype('category')
    
    # Converting dayofservice to datetime64
    x['DAYOFSERVICE'] = pd.to_datetime(x['DAYOFSERVICE'], infer_datetime_format=True)

    # Converting lastupdate to datetime64
    x['LASTUPDATE'] = pd.to_datetime(x['LASTUPDATE'], infer_datetime_format=True)
    
    return x

In [42]:
# Function to clean the trips df
# Based off of the CleaningNotebook cleaning methods
def clean_trips(x):
    categorical_columns = x[["TRIPID","LINEID", "ROUTEID", "DIRECTION", "PLANNEDTIME_ARR","PLANNEDTIME_DEP", "ACTUALTIME_ARR", "ACTUALTIME_DEP", "NOTE","JOURNEY_TIME"]].columns
    # Convert data type to category for these columns
    for column in categorical_columns:
        x[column] = x[column].astype('category')
    
    # Converting dayofservice to datetime64
    x['DAYOFSERVICE'] = pd.to_datetime(x['DAYOFSERVICE'], infer_datetime_format=True)

    # Converting lastupdate to datetime64
    x['LASTUPDATE'] = pd.to_datetime(x['LASTUPDATE'], infer_datetime_format=True)
    
    
    return x

In [9]:
# Function to clean the weather df
# Based off of the CleaningNotebook cleaning methods

def clean_weather(x):
    # Converting date to datetime
    x['date'] = pd.to_datetime(x['date'], infer_datetime_format=True)
    
    return x

### 2.0 Reading in Leavetimes Dataset

In [4]:
# Reading in the cleaned Leavetimes csv
df_leavetimes = pd.read_csv(r'C:\Users\jason\OneDrive - University College Dublin\Documents\MSc Computer Science\Summer Semester\Data\Notebooks\CleanedCSVs\rt_leavetimes_DB_2018_cleaned_sample.csv')

In [5]:
df_leavetimes.head(5)

,Unnamed: 0,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,SUPPRESSED,LASTUPDATE
0,0,2018-01-01 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,NaN,2018-01-08 17:21:10
1,1,2018-01-01 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,NaN,2018-01-08 17:21:10
2,2,2018-01-01 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,NaN,2018-01-08 17:21:10
3,3,2018-01-01 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284,NaN,2018-01-08 17:21:10
4,4,2018-01-01 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209,NaN,2018-01-08 17:21:10
...,...,...,...,...,...,...,...,...,...,...,...,...
116949108,116949108,2018-12-31 00:00:00,8588153,78,4383,28605,28605,28998,29013,3265721,NaN,2019-01-16 18:27:21
116949109,116949109,2018-12-31 00:00:00,8587459,78,4383,22695,22695,23247,23247,3265687,NaN,2019-01-16 18:27:21
116949110,116949110,2018-12-31 00:00:00,8586183,78,4383,51481,51481,52237,52283,2693229,NaN,2019-01-16 18:27:21
116949111,116949111,2018-12-31 00:00:00,8589374,23,7053,53659,53659,53525,53525,3265669,NaN,2019-01-16 18:27:21


In [6]:
# Pass the leavetimes df through the cleaning function
df_leavetimes = clean_leavetimes(df_leavetimes)
df_leavetimes.head(5)

,Unnamed: 0,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,SUPPRESSED,LASTUPDATE
0,0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,NaN,2018-01-08 17:21:10
1,1,2018-01-01,5966674,12,119,54001,54001,54023,54023,2693267,NaN,2018-01-08 17:21:10
2,2,2018-01-01,5959105,12,119,60001,60001,59955,59955,2693263,NaN,2018-01-08 17:21:10
3,3,2018-01-01,5966888,12,119,58801,58801,58771,58771,2693284,NaN,2018-01-08 17:21:10
4,4,2018-01-01,5965960,12,119,56401,56401,56309,56323,2693209,NaN,2018-01-08 17:21:10


In [7]:
df_leavetimes.dtypes

Unnamed: 0                  int64
DAYOFSERVICE       datetime64[ns]
TRIPID                   category
PROGRNUMBER              category
STOPPOINTID              category
PLANNEDTIME_ARR          category
PLANNEDTIME_DEP          category
ACTUALTIME_ARR           category
ACTUALTIME_DEP           category
VEHICLEID                category
SUPPRESSED               category
LASTUPDATE         datetime64[ns]
dtype: object

### 3.0 Read in weather CSV

In [15]:
#read in cleaned weather csc
df_weather = pd.read_csv(r'C:\Users\jason\OneDrive - University College Dublin\Documents\MSc Computer Science\Summer Semester\Data\Notebooks\CleanedCSVs\weather2018_cleaned.csv', index_col=[0])

In [17]:
df_weather.head(5)

,date,rain,temp,msl
0,2018-01-01 00:00:00,0.0,4.6,991.0
1,2018-01-01 01:00:00,0.1,4.7,991.1
2,2018-01-01 02:00:00,0.0,4.8,991.1
3,2018-01-01 03:00:00,0.0,4.9,990.7
4,2018-01-01 04:00:00,0.0,5.3,990.3


In [18]:
df_weather.dtypes

date     object
rain     object
temp    float64
msl     float64
dtype: object

In [19]:
# Sending the weather df to the cleaning function
df_weather = clean_weather(df_weather)
df_weather.head(5)

,date,rain,temp,msl
0,2018-01-01 00:00:00,0.0,4.6,991.0
1,2018-01-01 01:00:00,0.1,4.7,991.1
2,2018-01-01 02:00:00,0.0,4.8,991.1
3,2018-01-01 03:00:00,0.0,4.9,990.7
4,2018-01-01 04:00:00,0.0,5.3,990.3


In [20]:
df_weather.dtypes

date    datetime64[ns]
rain            object
temp           float64
msl            float64
dtype: object

### Merge Weather and LeaveTimes Dataset

In [21]:
# Sorting in order to merge
df_leavetimes.sort_values('LASTUPDATE', inplace=True)
df_weather.sort_values('date', inplace=True)

NameError: name 'df_leavetimes' is not defined

In [14]:
# Merging 
merged_data = pd.merge_asof(df_leavetimes, df_weather, left_on="LASTUPDATE", right_on="date")

In [15]:
merged_data.head(5)

,Unnamed: 0_x,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,SUPPRESSED,LASTUPDATE,Unnamed: 0_y,latitude,longitude,date,rain,temp,msl
0,0,2018-01-01,5972116,12,119,48030,48030,48012,48012,2693211,NaN,2018-01-08 17:21:10,185,53.364,-6.35,2018-01-08 17:00:00,0.0,3.7,1017.1
1,133482,2018-01-01,5970997,7,4095,40517,40517,40414,40435,2693273,NaN,2018-01-08 17:21:10,185,53.364,-6.35,2018-01-08 17:00:00,0.0,3.7,1017.1
2,133483,2018-01-01,5971005,7,4095,59688,59688,59621,59636,2693273,NaN,2018-01-08 17:21:10,185,53.364,-6.35,2018-01-08 17:00:00,0.0,3.7,1017.1
3,133484,2018-01-01,5971001,8,4192,50137,50137,50117,50117,2693273,NaN,2018-01-08 17:21:10,185,53.364,-6.35,2018-01-08 17:00:00,0.0,3.7,1017.1
4,133485,2018-01-01,5970999,8,4192,45337,45337,45376,45376,2693273,NaN,2018-01-08 17:21:10,185,53.364,-6.35,2018-01-08 17:00:00,0.0,3.7,1017.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116949108,116317939,2018-12-31,8591761,51,17,29109,29109,29559,29569,1000116,NaN,2019-01-16 18:27:21,8759,53.364,-6.35,2018-12-31 23:00:00,0.0,9.8,1035.2
116949109,116317940,2018-12-31,8580403,51,17,23160,23160,23424,23448,1932358,NaN,2019-01-16 18:27:21,8759,53.364,-6.35,2018-12-31 23:00:00,0.0,9.8,1035.2
116949110,116317941,2018-12-31,8576309,51,17,24060,24060,24232,24232,1001234,NaN,2019-01-16 18:27:21,8759,53.364,-6.35,2018-12-31 23:00:00,0.0,9.8,1035.2
116949111,116317935,2018-12-31,8585414,51,17,26942,26942,27554,27554,1000953,NaN,2019-01-16 18:27:21,8759,53.364,-6.35,2018-12-31 23:00:00,0.0,9.8,1035.2


### 4.0 Read In Trips Data for merging

In [43]:
#read in cleaned trips data
df_trips = pd.read_csv(r'C:\Users\jason\OneDrive - University College Dublin\Documents\MSc Computer Science\Summer Semester\Data\Notebooks\CleanedCSVs\rt_trips_DB_2018_cleaned.csv', index_col=[0])

In [44]:
df_trips.shape

(1893367, 11)

In [45]:
df_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE
0,1517961600000000000,6253783,68,68_80,1,87245,84600,87524.0,84600.0,1519819511000000000,",2967409,"
2,1517961600000000000,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,1519819511000000000,",2448968,"
3,1517961600000000000,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,1519819511000000000,",3094242,"
4,1517961600000000000,6253175,14,14_15,1,85383,81600,84682.0,81608.0,1519819511000000000,",2526331,"
5,1517961600000000000,6248240,77A,77A_28,2,41648,37200,42019.0,37538.0,1519819511000000000,",2966500,"


In [46]:
# df_trips = df_trips.drop(columns=["SUPPRESSED"])

In [47]:
# df_trips = df_trips.dropna()

In [48]:
df_trips.shape

(1893367, 11)

In [49]:
# Converting Actual time to int64 to subtract for new feature Journey Time. (Error on model when JOURNEY_TIME was left as a float)

df_trips['ACTUALTIME_ARR'] = df_trips['ACTUALTIME_ARR'].astype("int64")
df_trips['ACTUALTIME_DEP'] = df_trips['ACTUALTIME_DEP'].astype("int64")

In [50]:
# Creating an additional feature called Journey Time
df_trips['JOURNEY_TIME'] = df_trips['ACTUALTIME_ARR'] - df_trips['ACTUALTIME_DEP']

In [51]:
df_trips.dtypes

DAYOFSERVICE        int64
TRIPID              int64
LINEID             object
ROUTEID            object
DIRECTION           int64
PLANNEDTIME_ARR     int64
PLANNEDTIME_DEP     int64
ACTUALTIME_ARR      int64
ACTUALTIME_DEP      int64
LASTUPDATE          int64
NOTE               object
JOURNEY_TIME        int64
dtype: object

In [52]:
# Pass the leavetimes df through the cleaning function
df_trips = clean_trips(df_trips)
df_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME
0,2018-02-07,6253783,68,68_80,1,87245,84600,87524,84600,2018-02-28 12:05:11,",2967409,",2924
2,2018-02-07,6254942,45A,45A_70,2,35512,32100,36329,32082,2018-02-28 12:05:11,",2448968,",4247
3,2018-02-07,6259460,25A,25A_273,1,57261,54420,58463,54443,2018-02-28 12:05:11,",3094242,",4020
4,2018-02-07,6253175,14,14_15,1,85383,81600,84682,81608,2018-02-28 12:05:11,",2526331,",3074
5,2018-02-07,6248240,77A,77A_28,2,41648,37200,42019,37538,2018-02-28 12:05:11,",2966500,",4481


In [53]:
# df_trips = df_trips.replace(np.nan, '', regex=True)
# df_trips.head(5)

In [54]:
df_trips['MONTH'] = df_trips['DAYOFSERVICE'].dt.month
df_trips['DAY'] = df_trips['DAYOFSERVICE'].dt.dayofweek

In [55]:
df_trips.dtypes

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR          category
PLANNEDTIME_DEP          category
ACTUALTIME_ARR           category
ACTUALTIME_DEP           category
LASTUPDATE         datetime64[ns]
NOTE                     category
JOURNEY_TIME             category
MONTH                       int64
DAY                         int64
dtype: object

In [56]:
df_trips.sort_values(by=['TRIPID'])

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME,MONTH,DAY
681660,2018-01-06,5955221,14,14_15,1,42553,37800,42078,37836,2018-01-15 08:25:25,",2513511,",4242,1,5
723197,2018-01-06,5955222,14,14_16,2,47867,43200,47707,43182,2018-01-15 08:25:25,",2513514,",4525,1,5
684398,2018-01-06,5955223,14,14_15,1,54105,49200,54128,49171,2018-01-15 08:25:25,",2513515,",4957,1,5
719581,2018-01-06,5955224,14,14_16,2,60094,55200,60025,55203,2018-01-15 08:25:25,",2429419,",4822,1,5
674864,2018-01-06,5955225,14,14_15,1,65751,61200,65332,61161,2018-01-15 08:25:25,",2513518,",4171,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28895,2018-12-30,8592206,67,67_6,2,61896,58500,63920,58522,2019-01-16 18:16:31,",2805863,",5398,12,6
458924,2018-12-23,8592206,67,67_6,2,61896,58500,62418,58521,2019-01-08 17:30:40,",2805863,",3897,12,6
438572,2018-12-26,8592207,67,67_4,1,67655,63900,68012,63877,2019-01-09 17:30:44,",2805865,",4135,12,2
442075,2018-12-23,8592207,67,67_4,1,67655,63900,68707,63915,2019-01-08 17:30:40,",2805865,",4792,12,6


In [732]:
# merged_data.sort_values(by=['TRIPID'])

In [733]:
# merged_data2 = pd.merge(merged_data, df_trips, left_on="TRIPID", right_on="TRIPID")

In [734]:
# pd.set_option('display.max_columns', 999)

In [737]:
# Everything from route 68 from LeaveTimes
# Clean route 68 df
# Merging to weather and trips df
# Build a model from seq 1 to 2 ( eg. 119 to 44 etc)

### Practicing Modelling on route 68

In [60]:
## Creating a df based off of lineid 68
route68 = (df_trips[df_trips['LINEID'] =='68'])
route68.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME,MONTH,DAY
0,2018-02-07,6253783,68,68_80,1,87245,84600,87524,84600,2018-02-28 12:05:11,",2967409,",2924,2,2
78,2018-02-18,6263183,68,68_80,1,43305,40200,43313,40528,2018-02-26 11:09:33,",2967850,",2785,2,6
570,2018-06-26,7111764,68,68_81,1,60472,55800,59962,55858,2018-07-04 12:21:00,",3221763,",4104,6,1
593,2018-06-21,7023711,68,68_78,1,57296,52200,56558,52307,2018-06-29 15:43:00,",3225498,",4251,6,3
594,2018-06-21,7023760,68,68_83,2,50761,46800,50674,46851,2018-06-29 15:43:00,",3225496,",3823,6,3


In [61]:
# Sorting route68 by LASTUPDATE
route68.sort_values('LASTUPDATE', inplace=True)

In [62]:
# Merging weather and trips
merged_data_trips = pd.merge_asof(route68, df_weather, left_on="LASTUPDATE", right_on="date")

In [63]:
merged_data_trips.head(5)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,LASTUPDATE,NOTE,JOURNEY_TIME,MONTH,DAY,date,rain,temp,msl
0,2018-01-01,5969011,68,68_80,1,50690,47400,50073,47439,2018-01-08 17:21:10,",2967855,",2634,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
1,2018-01-01,5969018,68,68_80,1,87356,84600,87048,84675,2018-01-08 17:21:10,",2967986,",2373,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
2,2018-01-01,5969017,68,68_85,2,83636,81000,83405,81013,2018-01-08 17:21:10,",2967984,",2392,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
3,2018-01-01,5969008,68,68_85,2,40029,36600,39454,36662,2018-01-08 17:21:10,",2967848,",2792,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1
4,2018-01-01,5969012,68,68_80,1,62226,58800,61739,58796,2018-01-08 17:21:10,",2967859,",2943,1,0,2018-01-08 17:00:00,0.0,3.7,1017.1


In [65]:
merged_data_trips.dtypes

DAYOFSERVICE       datetime64[ns]
TRIPID                   category
LINEID                   category
ROUTEID                  category
DIRECTION                category
PLANNEDTIME_ARR          category
PLANNEDTIME_DEP          category
ACTUALTIME_ARR           category
ACTUALTIME_DEP           category
LASTUPDATE         datetime64[ns]
NOTE                     category
JOURNEY_TIME             category
MONTH                       int64
DAY                         int64
date               datetime64[ns]
rain                       object
temp                      float64
msl                       float64
dtype: object

In [66]:
# % missing on new feature JOURNEY_TIME
journey_missing = 100 * (merged_data_trips['JOURNEY_TIME'].isna().sum()/merged_data_trips.shape[0])
journey_missing

0.0

### Build and test Models

In [157]:
# X for the input data to be predicted on
X = merged_data_trips.drop(columns = ["JOURNEY_TIME","DAYOFSERVICE", "LASTUPDATE", "date","NOTE"])
# y for the out/goal set
y = merged_data_trips[["TRIPID"]]

In [158]:
# Split the dataset into two datasets: 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=1)

print("original range is: ", merged_data_trips.shape[0])
print("training range (70%):\t rows 0 to", round(X_train.shape[0]))
print("test range (30%): \t rows", round(X_train.shape[0]), "to", round(X_train.shape[0]) + X_test.shape[0])

original range is:  10132
training range (70%):	 rows 0 to 7092
test range (30%): 	 rows 7092 to 10132


In [162]:
X_train.head(5)

,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,MONTH,DAY,rain,temp,msl
3176,6769760,68,68_66,2,86700,84900,87340,85721,5,2,0.0,18.2,1008.9
9831,8460139,68,68_83,2,43655,39600,45441,39592,12,2,0.0,9.4,1025.2
8972,8068068,68,68_85,2,83350,80700,84522,80970,11,3,2.4,7.5,984.9
6529,7498472,68,68_80,1,43421,40200,43441,40197,8,6,0.0,12.1,1026.2
1224,6253776,68,68_83,2,50446,46800,51069,46834,2,2,0.0,-2.1,1018.3


In [159]:
X_train.dtypes

TRIPID             category
LINEID             category
ROUTEID            category
DIRECTION          category
PLANNEDTIME_ARR    category
PLANNEDTIME_DEP    category
ACTUALTIME_ARR     category
ACTUALTIME_DEP     category
MONTH                 int64
DAY                   int64
rain                 object
temp                float64
msl                 float64
dtype: object

In [164]:
multiple_linreg = LinearRegression().fit(X_train, y_train)

In [165]:
# Print the weights learned for each feature.
print("\nFeatures are: \n", X_train.columns)
print("\nCoeficients are: \n", multiple_linreg.coef_)
print("\nIntercept is: \n", multiple_linreg.intercept_)
print("\nFeatures and coeficients: \n", list(zip(X_train.columns, multiple_linreg.coef_)))


Features are: 
 Index(['TRIPID', 'LINEID', 'ROUTEID', 'DIRECTION', 'PLANNEDTIME_ARR',
       'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP', 'MONTH', 'DAY',
       'rain', 'temp', 'msl'],
      dtype='object')

Coeficients are: 
 [[ 1.00000000e+00 -1.67558716e-19  7.22543969e-14  1.71408927e-12
   1.44173724e-15 -2.41194665e-15  3.05612369e-16  7.09665784e-16
   6.45917096e-13 -1.15500924e-14  2.93161304e-13 -2.90107521e-13
   2.94646210e-14]]

Intercept is: 
 [-2.79396772e-09]

Features and coeficients: 
 [('TRIPID', array([ 1.00000000e+00, -1.67558716e-19,  7.22543969e-14,  1.71408927e-12,
        1.44173724e-15, -2.41194665e-15,  3.05612369e-16,  7.09665784e-16,
        6.45917096e-13, -1.15500924e-14,  2.93161304e-13, -2.90107521e-13,
        2.94646210e-14]))]
